In [48]:
import numpy as np
import qutip as qt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, Aer
from qiskit_aer import StatevectorSimulator
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import Statevector, random_statevector, partial_trace
from qiskit.circuit import Parameter
from qiskit.circuit.library import QFT
import random
from time import time
from typing import Optional

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')
from tools.quantum import *
from tools.classical import *


In [49]:
def energy_state_for_bits(bitstring: str) -> np.ndarray:
    """Convert a bitstring to an energy eigenstate."""
    tensor_list = []
    for bit in bitstring:
        if bit == '0':
            tensor_list.append(qt.basis(2, 0))
        else:
            tensor_list.append(qt.basis(2, 1))
    state = qt.tensor(tensor_list).full()
    return state / np.linalg.norm(state)


### QFT

In [50]:
# Circ
num_energy_bits = 6
bitstring = '100110'

some_initial_state = Statevector(energy_state_for_bits(bitstring))

circ = QuantumCircuit(num_energy_bits)
circ.initialize(some_initial_state, range(num_energy_bits))

qft = QFT(num_energy_bits)
circ.compose(qft, inplace=True)

circ_fourier_state = Statevector(circ).data.reshape(2**num_energy_bits, 1)


In [51]:
# Fourier computed on [0 - N-1], i.e. unsigned

all_bitstrings = [f'{i:0{num_energy_bits}b}' for i in range(2**num_energy_bits)]
basis_states_bin_ordered = [energy_state_for_bits(bitstring) for bitstring in all_bitstrings]

# bitstring_decimal = int(bitstring, 2)
# fourier_coeffs = np.zeros((2**num_energy_bits, 1), dtype=complex)
# for k in range(2**num_energy_bits):
#     fourier_coeffs[k] = np.exp(1j * 2 * np.pi * bitstring_decimal * k / 2**num_energy_bits) / np.sqrt(2**num_energy_bits)
    
# fourier_state = np.zeros((2**num_energy_bits, 1), dtype=complex)
# for k in range(2**num_energy_bits):
#     fourier_state += fourier_coeffs[k] * basis_states_bin_ordered[k]
    
# fourier_state = fourier_state / np.linalg.norm(fourier_state)

# distance = np.linalg.norm(circ_fourier_state - fourier_state)
# distance


In [54]:
# Fourier computed on [-N/2, N/2-1], i.e. signed

phase = int(bitstring[1:], 2)
if len(bitstring) != num_energy_bits:
    raise ValueError('This is not the right amount of phase bits')
if bitstring[0] == '1':
    phase = (int(bitstring[1:], 2) - 2**(num_energy_bits - 1))
else:
    phase = int(bitstring[1:], 2)
    
some_initial_state = Statevector(energy_state_for_bits(bitstring))

signed_interval = np.arange(-2**(num_energy_bits - 1), 2**(num_energy_bits - 1))
fourier_coeffs = np.zeros((2**num_energy_bits, 1), dtype=complex)
for i, k in enumerate(signed_interval):
    fourier_coeffs[i] = np.exp(1j * 2 * np.pi * phase * k / 2**num_energy_bits) / np.sqrt(2**num_energy_bits)
    
fourier_state = np.zeros((2**num_energy_bits, 1), dtype=complex)
for k in range(2**num_energy_bits):
    fourier_state += fourier_coeffs[k] * basis_states_bin_ordered[k]
    
fourier_state = fourier_state / np.linalg.norm(fourier_state)

distance = np.linalg.norm(circ_fourier_state - fourier_state)
distance

3.76422784870584e-15